In [9]:
import requests
import pandas as pd

In [10]:
api_key = 'key'

# 소환사 데이터 수집(Challenger, GrandMaster, Master)

In [11]:
def get_league_data(league):
    url = 'https://kr.api.riotgames.com/lol/league/v4/'+ league +'leagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
    league_df = pd.DataFrame(requests.get(url).json())
    league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
    league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
    league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합
    league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
#     league_df.to_csv('./data/' + league + '.csv',index=False,encoding = 'cp949')#중간저장
    return league_df

In [12]:
chal_df = get_league_data('challenger')

In [13]:
gma_df = get_league_data('grandmaster')

In [14]:
ma_df = get_league_data('master')

In [16]:
chal_df

,tier,freshBlood,hotStreak,inactive,leaguePoints,losses,summonerId,summonerName,veteran,wins
0,CHALLENGER,False,False,False,902,227,YAECZlT0gMbDeIkWn5VBAkVKULW9CzwqAep1fSNCPEmJyw,수기 김,False,286
1,CHALLENGER,False,False,False,1064,486,mFnLYwWsxr9WTQbCW0Iu2BfhjSO9AvxU2cyJeiUjwOjR1R8,매일배고프다,False,546
2,CHALLENGER,False,False,False,912,515,2RcQJSiSixrSgy5PavxekdovvtEAb_LwH2bLha1ibfI0NA,SANDBOX GorillA,True,561
3,CHALLENGER,False,True,False,1112,565,7uhS0N1zEjPJsO6SgU27CEfZ9950sL_d9X0rvhtNQhadiJg,SANDBOX Dove,True,615
4,CHALLENGER,False,False,False,1051,458,EEsreOggxSovcxK4vwqWERDl6eFi7H6DNg0soeRazgNdTGA,타 잔,True,536
5,CHALLENGER,False,False,False,925,369,5LxG__6rPJioqy3nZSmUvFx1NXexrg1bEsgJTJOFd5e4Nek,야 핑 좀 꺼줄래,False,427
6,CHALLENGER,True,False,False,980,351,xtJBtIEe0wXs8dhNaEEPB3B1HCmCrUEfzbus4nqXxpGg51M,T1 Cuzz,False,424
7,CHALLENGER,False,False,False,1074,754,GpbgHvXHm_ipUf2C04ftVDb_yASiJg5jgqpK0HJAkf18pto,Starryovo,False,838
8,CHALLENGER,False,False,False,1428,380,S6M62Yd79A96eYZGLsJXJ0Q11IcSA1y9xqCPznuugZx4pHg,잇츠 낫배드,True,485
9,CHALLENGER,True,False,False,773,1012,ZWWhJFXJmHRaUjBUKur3hUkMYRn9MFfgxLwoXzcGPCuVNw4,yiminggaoshou,False,1091


# 유저별 고유 accountId 수집

In [23]:
def get_accountId(league_df):
    for i in range(len(league_df)):
        try:
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)

            while r.status_code == 429:
                time.sleep(5)
                sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
                r = requests.get(sohwan)

            account_id = r.json()['accountId']
            league_df.iloc[i, -1] = account_id

        except:
            pass

In [24]:
get_accountId(chal_df)

In [25]:
chal_df.head()

,tier,freshBlood,hotStreak,inactive,leaguePoints,losses,summonerId,summonerName,veteran,wins
0,CHALLENGER,False,False,False,902,227,YAECZlT0gMbDeIkWn5VBAkVKULW9CzwqAep1fSNCPEmJyw,수기 김,False,Fe7sW62-hqr-VJxxtNV8R6ZrfXWze1kUPiJl-FQHAFi5
1,CHALLENGER,False,False,False,1064,486,mFnLYwWsxr9WTQbCW0Iu2BfhjSO9AvxU2cyJeiUjwOjR1R8,매일배고프다,False,i30QAALDUrwSriuiHTv7lOWb_SOXkvuf7vCW_KaQGlvC8W...
2,CHALLENGER,False,False,False,912,515,2RcQJSiSixrSgy5PavxekdovvtEAb_LwH2bLha1ibfI0NA,SANDBOX GorillA,True,x8Dvxgu_lCqWhWZ9zI4J7gFsvTRXnxYZwftL0onkVPGP
3,CHALLENGER,False,True,False,1112,565,7uhS0N1zEjPJsO6SgU27CEfZ9950sL_d9X0rvhtNQhadiJg,SANDBOX Dove,True,C8rfCYMVOvkVRsUrefEAXJsvfswNgTv66Brt0uwgzUYB6x0
4,CHALLENGER,False,False,False,1051,458,EEsreOggxSovcxK4vwqWERDl6eFi7H6DNg0soeRazgNdTGA,타 잔,True,qBp8jNLncDxENX8EzBrQWrMh6yhhn8Goy-45PkVFbCN23BU
